# 1. Installing Dependencies

!pip install -q datasets transformers accelerate timm
!pip install -q -U albumentations>=1.4.5 torchmetrics pycocotools

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim import Adam, SGD
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms
from torchvision.models import models

ImportError: cannot import name 'models' from 'torchvision.models' (/Users/darky/Documents/Q-Vision/base/lib/python3.12/site-packages/torchvision/models/__init__.py)

# Building and Playing with DATA!

from roboflow import Roboflow
rf = Roboflow(api_key="here")
project = rf.workspace("roboflow-100").project("uno-deck")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
def compare(img_path):

  """
    Compare different sizes of YOLOv8 models for Uno.
    
    Parameters:

    image_path : str
        Path to the input image for object detection
    
    Returns:

    dict
        A dictionary containing detection results and performance metrics 
        for each model size
  """
    model_sizes = ['n','s','m','l']
  img = cv2.imread(img_path)

  detection_results = {} # -> dictionary to store results

  for size in model_sizes:

    model = YOLO(f'yolov8{size}.pt')
    
    start = time.time()

    results = model(img)
    end = time.time() - start

    result = results[0]
    
    detection_results[size] = {
            'inference_time': end,
            'num_detections': len(result.boxes),
            'confidence_scores': result.boxes.conf.tolist() if len(result.boxes) > 0 else [],
            'classes': result.boxes.cls.tolist() if len(result.boxes) > 0 else []
        }
    
    annotated_frame = result.plot()
    cv2.imwrite(f'yolov8{size}_result.jpg', annotated_frame)
    
  return detection_results

In [ ]:
def analyze(results):
    """
    Analyze and compare the detection results across different model sizes.
    
    Parameters:
    -----------
    results : dict
        Dictionary of detection results from different YOLO model sizes
    
    Returns:
    --------
    None
        Prints a detailed comparison and creates visualizations
    """
    model_sizes = list(results.keys())
    inference_time = [results[size]['inference_time'] for size in model_sizes]
    num_detections = [results[size]['num_detections'] for size in model_sizes]

    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (12,5))

    ax1.bar(model_sizes, inference_time)
    ax2.set_title('Inference time comparision')
    ax2.set_xlabel('Model')
    ax2.set_ylabel('Inf time')

    ax2.bar(model_sizes, num_detections)
    ax2.set_title('Number of detections')
    ax2.set_xlabel('Model')
    ax2.set_ylabel('Num of Detections')